Обучение происходило на Google Colab c Tesla T4

In [172]:
!nvidia-smi

Tue Aug  2 11:46:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Первичные фичи, которые могут понадобится в регрессоре
1. Количество всех парамметров
2. Суммарное кол-во параметров каждого типа слоя
3. Количесвто слоев по категориям

In [173]:
data = {
    'all_params': [],
    'conv_n_params': [], 
    'conv_batch_n_params': [], 
    'linear_batch_n_params': [],  
    'dropout_p': [],
    'linear_n_params': [],
    'conv_layers': [], 
    'pool_layers': [],
    'conv_batch_layers': [], 
    'linear_batch_layers': [],  
    'dropout_layers': [],
    'linear_layers': [],
    "time": []
} 

In [174]:
import torch
import torch.nn as nn
from torchvision import models

import pandas as pd 
import random

from tqdm import tqdm
from time import time

В модуле nn есть огромное множество разновидностей каждого типа слоя. Можно пробовать их комбинировать, наблюдать за скоростью обучения при разных типах слоев. Но изначально я поставил себе задачу генерации самого обычного семейства сверточных сетей. 

In [ ]:
dir(nn) 

С самого начала я рассчитывал, что буду прогонять генератор на размерностях от 32 до 512, однако, тогда размерность входного датасета будет сильно меняться (а мы должны создавать одинаковые условия), а если зафиксировать размерность входного датасета, например, на 512, то у нейронной сети, рассчитнной на размерность 32, будет большое число параметров при переходе к линейному слою, что значительно увеличит время обучения. Из-за этого пришлось зафиксировать входную размерность на 256, и делать расчёт кол-ва слоев в соответствии с размерностью.

In [176]:
N_MODELS = 100
INPUT_SIZE = 256
BATCH_SIZE = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

kernel_sizes = [1, 3, 5]
paddings = [0, 1, 3]
strides = [1, 2, 3]

Генерация датасета из 200 * 128 объектов заданной размерности

In [177]:
train_loader = []
for i in range(200):
    train_batch = torch.rand((BATCH_SIZE,1,INPUT_SIZE,INPUT_SIZE))
    train_labels = torch.randint(0, 10, (1, BATCH_SIZE)).reshape(-1)
    train_loader.append((train_batch, train_labels))

Генератор создает CNN модели в зависимости от указываемого размера входного тензора. 

Атрибуты класса:
1. layers - список всех слоев в модели
2. data - словарь для записи описания модели

В методе init() расположены два цикла. Первый отвечает за добавление слоев в сверточную часть, а второй - в линейную. Парамметры для слоев выбираются случайным образом из заготовленных списков. В методе forward при помощи итерации по всем слоям реализуется алгоритм прямого расспростронения ошибки.



In [178]:
class Net(nn.Module):

    def __init__(self, input_size):
        super().__init__()

        init = True # флаг для инициализации первого слоя через nn.ModuleList
        self.layers = None
        self.data = {
            'all_params': 0,
            'conv_n_params': 0, 
            'conv_batch_n_params': 0, 
            'linear_batch_n_params': 0, 
            'dropout_p': 0,
            'linear_n_params': 0,
            'conv_layers': 0, 
            'pool_layers': 0,
            'conv_batch_layers': 0, 
            'linear_batch_layers': 0, 
            'dropout_layers': 0,
            'linear_layers': 0,
            "time": 0
        } 

        n_pool_layers = random.randint(2, 4) # кол-во слоев пуллинга
        self.data['pool_layers'] = int(n_pool_layers)

        # число каналов
        in_channels = 1 
        out_channels = 16

        # генерация случайных значений из равномерного распределения для реализации случайного добавления слоев
        p_gen = random.uniform(0,1)
        p_batchnorm = random.uniform(0,1)
        p_dropout = random.uniform(0,1)

        p = torch.randint(2, 5, (1,1)) / 10 # параметр для dropout
        self.data['dropout_p'] = round(float(p), 2)

        while input_size != 1:

            kernel_size = kernel_sizes[random.randint(0, len(kernel_sizes)-1)]
            stride = strides[random.randint(0, len(strides)-1)]
            padding = paddings[random.randint(0, len(strides)-1)]
            output_size = (input_size + 2 * padding - kernel_size) / stride + 1

            if input_size > kernel_size:
                if init:
                    self.layers = nn.ModuleList([nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)])
                    init = False
                else:
                    self.layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding))
                
                self.data['conv_layers'] += 1
                self.data['conv_n_params'] += (kernel_size * kernel_size * in_channels + 1) * out_channels # подсчет кол-ва парамметров сверточных слоев

                if (p_gen <= p_batchnorm):
                    self.layers.append(nn.BatchNorm2d(out_channels))
                    self.data['conv_batch_layers'] += 1
                    self.data['conv_batch_n_params'] += out_channels * 2 # подсчет кол-ва парамметров сверточных батчнорм слоев

                self.layers.append(nn.ReLU(True))
                input_size = output_size

                if n_pool_layers > 0 and input_size > 2:
                    self.layers.append(nn.MaxPool2d(2,2))
                    output_size /= 2
                    n_pool_layers -= 1
                    input_size = output_size
                
                in_channels = out_channels
                if input_size < 32:
                    out_channels = 256
                elif input_size < 64:
                    out_channels = 128
                elif input_size < 128:
                    out_channels = 64
                else:
                    out_channels = 32
            else:
              break

        # преобразование входной размерности к одному измерению
        input_size = int(input_size) * int(input_size) * in_channels

        n_lin_layers = torch.randint(1, 3, (1,1)) # число линейных слоев
        self.data['linear_layers'] = int(n_lin_layers)
        self.layers.append(nn.Flatten())

        while(n_lin_layers > 1):
            self.layers.append(nn.Linear(input_size, input_size))
            self.data['linear_n_params'] += (input_size + 1) * input_size # подсчет кол-ва парамметров линейных слоев
            
            if (p_gen <= p_batchnorm):
                self.layers.append(nn.BatchNorm1d(input_size))
                self.data['linear_batch_layers'] += 1
                self.data['linear_batch_n_params'] += input_size * 2 # подсчет кол-ва парамметров batchnorm слоев
            
            self.layers.append(nn.ReLU(True))
            if (p_gen <= p_dropout):
                self.layers.append(nn.Dropout(round(float(p),2)))
                self.data['dropout_layers'] += 1
            n_lin_layers -= 1

        self.layers.append(nn.Linear(input_size, 1000))
        self.data['linear_n_params'] += (input_size + 1) * 1000 # подсчет кол-ва парамметров линейных слоев

        self.data['pool_layers'] -= int(n_pool_layers) # если входная размерность слишком мала и не все были задействованы

    def forward(self, x):
        for i in range(len(self.layers)):
            x = self.layers[i](x)
        return x

Цикл создания моделей и сбора датасета по трем замерам времени для каждого объекта на одной эпохе. При знании кол-ва эпох рассчитывается полное время обучения

In [ ]:
for i in range(N_MODELS):

    model = Net(INPUT_SIZE).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.11) # так сложилось, что был выбран SGD
    loss_function = nn.CrossEntropyLoss() # так сложилось, что была выбрана кроссэнтропия, 
                                          # но это не имеет никакого значения, потому что не решается никая задача

    model.train()
    common_time = 0
    for test in range(3):
        start = time()
        for i, (batch, labels) in enumerate(tqdm(train_loader)):
            loss = torch.tensor(0, dtype=torch.float32)
            optimizer.zero_grad()
            batch = batch.to(device)
            labels = labels.to(device)
            output = model(batch)
            loss = loss_function(output, labels)
            loss.backward()
            optimizer.step()

        common_time += time()-start

    common_time = round(float(common_time / 3), 3)
    model.data['time'] = common_time
    model.data['all_params'] = sum([p.numel() for p in model.parameters() if p.requires_grad]) # подсчет всех обучаемых параметров

    for key in data:
        data[key].append(model.data[key])
    print(N_MODELS - i)

Итоговый датасет

In [187]:
df = pd.read_csv("/content/out.csv") # ваш путь
df

,all_params,conv_n_params,conv_batch_n_params,linear_batch_n_params,dropout_p,linear_n_params,conv_layers,pool_layers,conv_batch_layers,linear_batch_layers,dropout_layers,linear_layers,time
0,2967039,85409,0,0,0.4,2881630,9,2,0,0,0,2,15.441
1,404086,27086,0,0,0.4,377000,8,2,0,0,0,1,3.471
2,388291,9057,234,0,0.2,379000,6,4,6,0,0,1,3.121
3,46189,17039,150,0,0.3,29000,5,4,5,0,0,1,5.569
4,209205,205,0,0,0.3,209000,3,2,0,0,0,1,2.700
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15371,1189,0,0,0.2,14182,3,3,0,0,1,2,2.077
96,262685,9535,150,0,0.3,253000,5,2,5,0,0,1,5.529
97,898788,7238,360,1134,0.4,890056,7,4,7,1,1,2,19.857
98,11234659,290907,1884,5688,0.2,10936180,11,3,11,1,1,2,5.312


Признаки all_params,	conv_n_params,	conv_batch_n_params имеют сильную корреляцию. Самый высокий коэффициент корреляциии с предикатом у conv_n_params, поэтому именно его было принято оставить.

In [188]:
df.corr()

,all_params,conv_n_params,conv_batch_n_params,linear_batch_n_params,dropout_p,linear_n_params,conv_layers,pool_layers,conv_batch_layers,linear_batch_layers,dropout_layers,linear_layers,time
all_params,1.000000,0.923351,0.833270,0.320155,0.085366,0.482071,0.540488,-0.176471,0.448579,0.052238,0.041130,-0.003167,0.844137
conv_n_params,0.923351,1.000000,0.834727,-0.010617,0.116309,0.108723,0.422831,-0.134267,0.379917,-0.038233,-0.042921,-0.086062,0.902245
conv_batch_n_params,0.833270,0.834727,1.000000,0.222362,0.045453,0.252303,0.668508,-0.200030,0.698571,0.156119,0.120654,-0.004144,0.753447
linear_batch_n_params,0.320155,-0.010617,0.222362,1.000000,-0.065880,0.852875,0.325016,-0.078209,0.447693,0.525591,0.405295,0.302566,0.009336
dropout_p,0.085366,0.116309,0.045453,-0.065880,1.000000,-0.044335,0.026789,0.057279,-0.056929,0.086400,-0.039331,-0.014460,0.122706
linear_n_params,0.482071,0.108723,0.252303,0.852875,-0.044335,1.000000,0.434249,-0.150466,0.293996,0.222152,0.204179,0.188072,0.126709
conv_layers,0.540488,0.422831,0.668508,0.325016,0.026789,0.434249,1.000000,-0.298955,0.556596,0.167566,0.137533,0.075107,0.463864
pool_layers,-0.176471,-0.134267,-0.200030,-0.078209,0.057279,-0.150466,-0.298955,1.000000,-0.198339,-0.104874,-0.063925,-0.082880,-0.041417
conv_batch_layers,0.448579,0.379917,0.698571,0.447693,-0.056929,0.293996,0.556596,-0.198339,1.000000,0.542305,0.349897,0.102237,0.363639
linear_batch_layers,0.052238,-0.038233,0.156119,0.525591,0.086400,0.222152,0.167566,-0.104874,0.542305,1.000000,0.700067,0.575669,-0.027804


In [189]:
x = df
timing = df['time']
del x['time']
del x['all_params']
del x['conv_batch_n_params']
x

,conv_n_params,linear_batch_n_params,dropout_p,linear_n_params,conv_layers,pool_layers,conv_batch_layers,linear_batch_layers,dropout_layers,linear_layers
0,85409,0,0.4,2881630,9,2,0,0,0,2
1,27086,0,0.4,377000,8,2,0,0,0,1
2,9057,0,0.2,379000,6,4,6,0,0,1
3,17039,0,0.3,29000,5,4,5,0,0,1
4,205,0,0.3,209000,3,2,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
95,1189,0,0.2,14182,3,3,0,0,1,2
96,9535,0,0.3,253000,5,2,5,0,0,1
97,7238,1134,0.4,890056,7,4,7,1,1,2
98,290907,5688,0.2,10936180,11,3,11,1,1,2


In [190]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [191]:
scaler = StandardScaler()
scaler.fit(x)
scaled_df = scaler.transform(x)

X_train, X_test, y_train, y_test = train_test_split(scaled_df, 
                                                    timing, 
                                                    test_size=0.33, 
                                                    random_state=42)

В качестве модели регрессора использовал линейную регрессию, так как она лучше всего себя показа по сравнению со случайным лесом и другими моделями. Самая высокая важность признака у conv_n_params. Также есть влияние количества разных типов слоев.

In [192]:
linear_regression_model = SGDRegressor() 
linear_regression_model.fit(X_train, y_train)
linear_regression_model.coef_

array([ 7.61670679, -0.31219127,  0.18405429, -0.17252001,  1.17778492,
        1.21697881,  0.94144176, -0.36450784,  0.40594042, -0.15905383])

После долгих проб и ошибок, пришло осознание, что необходимы эксперименты по извлечению новых признаков (очень сложно дается судить, чтоже все-таки влияет на скорость обучения), увеличение объема датасета и усовершенствованию генератора моделей.

In [193]:
train_predictions = linear_regression_model.predict(X_train) 
test_predictions = linear_regression_model.predict(X_test)

train_mae = mean_absolute_error(y_train, train_predictions)
test_mae = mean_absolute_error(y_test, test_predictions)

train_mape = mean_absolute_percentage_error(y_train, train_predictions)
test_mape = mean_absolute_percentage_error(y_test, test_predictions)

print("Train MAE: {}".format(train_mae))
print("Test MAE: {}".format(test_mae))
print("Train MAPE: {}".format(train_mape))
print("Test MAPE: {}".format(test_mape))

Train MAE: 2.917827589112228
Test MAE: 3.2399154316531193
Train MAPE: 0.5697522814472361
Test MAPE: 0.674596019932665


In [165]:
df.to_csv('out.csv', index=False) 